# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,badger,64.8000,-147.5333,21.14,51,75,3.09,US,1718050801
1,1,punta arenas,-53.1500,-70.9167,2.06,93,75,1.54,CL,1718050802
2,2,isafjordur,66.0755,-23.1240,6.07,75,100,2.43,IS,1718050802
3,3,petropavlovsk-kamchatsky,53.0452,158.6483,12.33,86,0,4.47,RU,1718050803
4,4,ribeira grande,38.5167,-28.7000,20.43,77,40,3.60,PT,1718050803


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:


# Configure the map plot
map_plot_1 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    size = "Humidity",
    geo = True,
    tiles = "OSM",
    color = "City",
    alpha = 0.35,
    frame_width = 700,
    frame_height = 500,
    )
# Display the map
map_plot_1



C:\Users\User\anaconda3\envs\dev\lib\site-packages\paramiko\transport.py:219: CryptographyDeprecationWarning: Blowfish has been deprecated and will be removed in a future release
  "class": algorithms.Blowfish,


:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
filtered_city_data_df = city_data_df[(city_data_df['Humidity'] < 50) & (city_data_df['Max Temp'] < 30) & (city_data_df['Wind Speed'] < 20)]

# Drop any rows with null values
filtered_city_data_df = filtered_city_data_df.copy()
filtered_city_data_df.dropna(inplace=True)

# Display sample data
filtered_city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
35,35,baotou,40.6522,109.8222,23.17,32,29,0.96,CN,1718050807
42,42,thompson,55.7435,-97.8558,20.09,34,75,4.12,CA,1718050808
54,54,menongue,-14.6585,17.6910,13.79,45,0,2.21,AO,1718050809
68,68,queenstown,-31.8976,26.8753,8.76,47,0,2.33,ZA,1718050811
85,85,laojunmiao,39.8333,97.7333,18.27,30,47,4.75,CN,1718050813


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = filtered_city_data_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ""

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
35,baotou,CN,40.6522,109.8222,32,
42,thompson,CA,55.7435,-97.8558,34,
54,menongue,AO,-14.6585,17.6910,45,
68,queenstown,ZA,-31.8976,26.8753,47,
85,laojunmiao,CN,39.8333,97.7333,30,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories": "accommodation.hotel",
    "limit":1,
    "apiKey":geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row['Lat']
    lon = row['Lng']
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lon},{lat},{radius}"
    params["bias"] = f"proximity:{lon},{lat}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url,params=params)
    
    # Convert the API response to JSON format
    name_address = requests.get(base_url,params=params).json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
baotou - nearest hotel: 包头宾馆
thompson - nearest hotel: Thompson Inn
menongue - nearest hotel: No hotel found
queenstown - nearest hotel: Queens Hotel
laojunmiao - nearest hotel: No hotel found
saldanha - nearest hotel: Hoedjesbaai
whitehorse - nearest hotel: Town & Mountain Hotel
arraial do cabo - nearest hotel: No hotel found
tongliao - nearest hotel: No hotel found
tarinkot - nearest hotel: No hotel found
lepsy - nearest hotel: Orley
apolinario saravia - nearest hotel: No hotel found
yanykurgan - nearest hotel: No hotel found
udachny - nearest hotel: Вилюй
port augusta - nearest hotel: Hotel Commonwealth
cuito - nearest hotel: No hotel found
cadillac - nearest hotel: Holiday Inn
ust-nera - nearest hotel: гостиница "Солнечная"
ravar - nearest hotel: زائر سرا راور
kapuskasing - nearest hotel: Travelodge
vernon - nearest hotel: Okanagan Royal Park Inn
urumqi - nearest hotel: 百时快捷酒店红山店
berryville - nearest hotel: No hotel found
gilgit - nearest hotel: PTDC Motel
ple

,City,Country,Lat,Lng,Humidity,Hotel Name
35,baotou,CN,40.6522,109.8222,32,包头宾馆
42,thompson,CA,55.7435,-97.8558,34,Thompson Inn
54,menongue,AO,-14.6585,17.6910,45,No hotel found
68,queenstown,ZA,-31.8976,26.8753,47,Queens Hotel
85,laojunmiao,CN,39.8333,97.7333,30,No hotel found
...,...,...,...,...,...,...
544,kalifornsky,US,60.4183,-151.2900,46,No hotel found
550,sierra grande,AR,-41.6060,-65.3557,42,Sierra Grande
565,balykshi,KZ,47.0667,51.8667,41,Арасан
575,zhangatas,KZ,43.5667,69.7500,30,Арман


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [7]:
%%capture --no-display

# Configure the map plot
map_plot = hotel_df.hvplot.points('Lng', 'Lat', size='Humidity', color='blue', hover_cols=['City', 'Country', 'Hotel Name'])

# Display the map
map_plot

:Points   [Lng,Lat]   (Humidity,City,Country,Hotel Name)